In [1]:
import json
import os
import time
from uuid import uuid4

import numpy as np
import redis
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image
import tensorflow as tf

2024-01-04 17:52:12.492984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-04 17:52:12.493040: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
REDIS_QUEUE = "service_queue"
REDIS_PORT = 6379
REDIS_DB_ID = 0
REDIS_IP = os.getenv("REDIS_IP", "redis")
SERVER_SLEEP = 0.05

db = redis.Redis(
    host=REDIS_IP,
    port=REDIS_PORT,
    db=REDIS_DB_ID
)

model = ResNet50(include_top=True, weights="imagenet")

2024-01-04 17:52:19.924592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-04 17:52:19.924691: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-04 17:52:19.924752: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9105339d2bc3): /proc/driver/nvidia/version does not exist
2024-01-04 17:52:19.925294: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [93]:
for i in range(10):
    job_id = f"{uuid4()}"
    job_data = dict(id=f"{job_id}", image_name="dog.jpeg")
    db.lpush(REDIS_QUEUE,  json.dumps(job_data))

In [94]:
batch_size = 10
filenames = []
ids = []
for i in range(batch_size):
    queue_value = db.lpop("service_queue")
    if queue_value is None:
        break
    data = json.loads(queue_value.decode())
    filename = data.get("image_name")
    id = data.get("id")
    ids.append(id)
    filenames.append(filename)
# filenames, ids

In [95]:
def load_image(filename):
    img = image.load_img(img_path, target_size=(224, 224))
    return image.img_to_array(img)

In [96]:
filenames = [os.path.join(".", filename) for filename in filenames]
dataset = np.array([load_image(filename) for filename in filenames])
dataset_batch = preprocess_input(dataset)
preds = model.predict(dataset_batch, batch_size=batch_size)
preds = decode_predictions(preds, top=1)

In [97]:
for pred, job_id in zip(preds, ids):
    _, class_name, pred_probability = pred[0]
    result = dict(prediction=class_name,
                  score=float(f"{pred_probability:.4f}"))
    db.set(job_id, json.dumps(result))

In [100]:
db.get(ids[4])

b'{"prediction": "Eskimo_dog", "score": 0.9346}'

In [99]:
ids

['b1fa2127-7e55-4130-91de-6b72bedeeb2f',
 'f54b0ef4-e30c-47f9-84f8-622993c60770',
 'f79e2aca-0d3d-4acd-acdd-81d8a7c9360c',
 '9ee9d23e-ff99-4ba6-a62b-52878b463fb1',
 '2a442eb9-2b4e-45d2-9d12-509617c00b5d',
 '5fb46d6b-db80-484c-ab6d-4f90da3da916',
 '4f35d2c8-db71-44c8-b6ae-1c9b04f97e20',
 '752318c6-f882-44ce-b449-9696d5f0f603',
 '53bf2525-65e3-4bea-bc47-73ab646e4dfa',
 'e2225055-db84-412e-844f-3d233a5e1160']